In [1]:
import numpy as np
from astropy import units
from astropy.coordinates import Angle

from bda import decorr_calc as dc

In [4]:
# define correlator specifics
max_decorr = 0.1  # acceptable decorrelation fraction
freq = 250 * units.MHz
corr_FoV_angle = Angle(20.0 * units.deg)
chan_width = (250 * units.MHz / 8192)  # 250 MHz bandwidth over 8192 channels
pre_fs_int_time = 0.1 * units.s  # 100 ms fringe stopping inside correlator
corr_int_time = 2 * units.s  # default correlator integration time
max_int_time = 16 * units.s  # maximum integration time permitted

# figure out maximum number of samples to average
max_samples = (max_int_time / corr_int_time).to(units.dimensionless_unscaled)
max_two_foldings = int(np.floor(np.log2(max_samples)))

In [15]:
h3c_ants_fn = "/Users/plaplant/Documents/school/penn/software/ProjectFiles/spec_calcs/h2c/hera_350.dat"
ant_names = np.loadtxt(h3c_ants_fn, usecols=(0,), dtype=str)
ant_nums = np.array([int(ant_name[2:]) for ant_name in ant_names])
ant_pos = np.loadtxt(h3c_ants_fn, usecols=(1, 2, 3))

# antennas positions are technically eastings/northings, but close enough to treat as ENU
N_ants = len(ant_nums)
num_baselines = int(N_ants * (N_ants + 1) / 2)
ant_1_array = np.zeros((num_baselines,), dtype=np.int)
ant_2_array = np.zeros((num_baselines,), dtype=np.int)
uvw_array = np.zeros((num_baselines, 3))
bda_compression = np.zeros((num_baselines,))
bl = 0
for i in range(N_ants):
    for j in range(i, N_ants):
        ant_1_array[bl] = i
        ant_2_array[bl] = j
        uvw_array[bl, :] = ant_pos[i, :] - ant_pos[j, :]

        # compute expected decorr
        lx = np.abs(uvw_array[bl, 0]) * units.m
        ly = np.abs(uvw_array[bl, 1]) * units.m

        if i == j:
            # autocorrelation
            n_two_foldings = 0
        else:
            # figure out BDA factor
            n_two_foldings = dc.bda_compression_factor(
                max_decorr,
                freq,
                lx,
                ly,
                corr_FoV_angle,
                chan_width,
                pre_fs_int_time,
                corr_int_time
            )
        n_two_foldings = min(n_two_foldings, max_two_foldings)

        # save result in array
        bda_compression[bl] = 2**(-n_two_foldings)

        # increment baseline number
        bl += 1

# define default parameters
nchans_default = 6144
npols_default = 4
hours_default = 12

base_compression = np.sum(bda_compression)
base_volume = 60 * units.TB

# Observation Options
Change the options below to understand the impact on resulting data volumes.

In [18]:
# general observing options
nchans_to_keep = 6144
npols_to_keep = 4
hours_to_keep = 12
keep_diffs = True

# BDA options
decorr_level = 0.1
decorr_FoV_angle = Angle(20.0 * units.deg)
aggressive_int_time = 16 * units.s
max_samples = (aggressive_int_time / corr_int_time).to(units.dimensionless_unscaled)
max_two_foldings = int(np.floor(np.log2(max_samples)))

# baseline cut options
bl_len_min = 0.0 * units.m
bl_len_max = 1 * units.km

# figure out which baselines to keep
bls_kept = np.ones_like(bda_compression, dtype=np.bool)

for i, comp_factor in enumerate(bda_compression):
    # see if bl length is in range
    bl_len = np.linalg.norm(uvw_array[i, :]) * units.m
    if bl_len < bl_len_min or bl_len > bl_len_max:
        bls_kept[i] = False

# figure out resulting volume
chan_frac = nchans_to_keep / nchans_default
pol_frac = npols_to_keep / npols_default
time_frac = hours_to_keep / hours_default

chan_width /= chan_frac

if (decorr_level > max_decorr or decorr_FoV_angle > corr_FoV_angle
    or aggressive_int_time > max_int_time):
    # we need to recompute the bda_compression factor
    new_bda_comprression = np.zeros_like(bda_comprssion)
    for bl in range(len(bda_compression)):
        if not bls_kept[bl]:
            continue
        i = ant_1_array[bl]
        j = ant_2_array[bl]
        lx = uvw_array[bl, 0] * units.m
        ly = uvw_array[bl, 1] * units.m
        if i == j:
            n_two_foldings = 0
        else:
            # figure out BDA factor
            n_two_foldings = dc.bda_compression_factor(
                decorr_level,
                freq,
                lx,
                ly,
                decorr_FoV_angle,
                chan_width,
                pre_fs_int_time,
                corr_int_time
            )
        n_two_foldings = min(n_two_foldings, max_two_foldings)
        
        # save result in array
        bda_compression[bl] = 2**(-n_two_foldings)
else:
    new_bda_compression = bda_compression
    
masked_bda = np.ma.masked_array(new_bda_compression, mask=~bls_kept)
bda_frac = np.ma.sum(masked_bda) / np.sum(bda_compression)

In [19]:
# Grand total
reduction_frac = 1 * chan_frac * pol_frac * time_frac * bda_frac
if not keep_diffs:
    reduction_frac *= 0.5

print("New nightly data volume: ")
print(reduction_frac * base_volume)

New nightly data volume: 
60.0 Tbyte
